In [1]:
import os
import os.path as osp
from glob import glob
import re
from pprint import pprint
from tqdm import tqdm
import pydicom as dicom

PATHS = [f'/kuacc/users/oince22/hpc_run/physionet.org/files/mimic-cxr-jpg/2.0.0/files/p{s}' for s in range(10, 13)]
TEXT_PATH_ROOT = '/datasets/mimic/physionet.org/files/mimic-cxr/2.0.0/files/'

get_report_path = lambda img_path: Path.joinpath(TEXT_ROOT, img_path.parents[0]).with_suffix(".txt")

def check_jpg_extension(filename):
    pattern = r"\.jpg$"
    if re.search(pattern, filename):
        return True
    else:
        return False

In [2]:
ALL_IMG_PATHS = []
ALL_TXT_PATHS = []

In [3]:
get_txt_path = lambda s: TEXT_PATH_ROOT + "/".join(s.split("/")[10:-1]).split(".")[0] + ".txt"
get_new_img_path = lambda s: "/".join(s.split("/")[10:])

In [4]:
for PATH in PATHS:
    top_dirs = [dir for dir in os.listdir(PATH) if dir.find(".") == -1]
    for dir_name in tqdm(top_dirs):
        paths = os.listdir(osp.join(PATH, dir_name))
        img_dirs = [osp.join(PATH, dir_name, dir) for dir in paths if dir.find(".") == -1]
        for img_dir in img_dirs:
            img_paths = [osp.join(img_dir, img_name) for img_name in os.listdir(img_dir)]
            img_paths = [path for path in img_paths if check_jpg_extension(path)]
            ALL_IMG_PATHS.extend([get_new_img_path(s) for s in img_paths])
            ALL_TXT_PATHS.extend([get_txt_path(s) for s in img_paths])

100%|██████████| 6526/6526 [00:18<00:00, 355.18it/s]


In [5]:
import string
def preprocess_report(text):
    # Remove unnecessary and insensible parts
    text = re.sub(r"EXAMINATION:.*", "", text)  # Remove EXAMINATION line
    text = re.sub(r"WET READ:.*", "", text)  # Remove WET READ line
    text = re.sub(r"FINAL REPORT", "", text)  # Remove FINAL REPORT line
    text = re.sub(r"STUDY:.*", "", text)  # Remove STUDY line
    text = re.sub(r"COMPARISON:.*", "", text)  # Remove COMPARISON section
    text = re.sub(r"TECHNIQUE:.*", "", text)  # Remove TECHNIQUE section
    text = re.sub(r"_+", "_", text)  # Remove multiple underscores

    # Clean up excessive newlines and spaces
    text = re.sub(r"\s\s+", " ", text)
    text = re.sub("[^a-zA-Z0-9 :.,]", "", text)
    text = re.sub(r" +", " ", text)
    text = text.strip()
    return text

In [10]:
import csv

def create_tsv():
    with open(f"MIMIC_JPG.tsv", 'w') as f:
        writer = csv.writer(f, delimiter='\t')
        writer.writerow(["caption", "image"])
        for txt_path, img_path in tqdm(zip(ALL_TXT_PATHS, ALL_IMG_PATHS), total=len(ALL_TXT_PATHS)):
            try:
                f = open(txt_path, "r")
            except FileNotFoundError as e:
                print(str(e))
                continue
            txt = preprocess_report(f.read().strip())
            f.close()
            writer.writerow([txt, img_path])

In [11]:
create_tsv()

 76%|███████▋  | 85815/112413 [01:30<00:30, 874.74it/s] 

[Errno 2] No such file or directory: '/datasets/mimic/physionet.org/files/mimic-cxr/2.0.0/files/p12/p12073186/s51461779.txt'
[Errno 2] No such file or directory: '/datasets/mimic/physionet.org/files/mimic-cxr/2.0.0/files/p12/p12073186/s51461779.txt'
[Errno 2] No such file or directory: '/datasets/mimic/physionet.org/files/mimic-cxr/2.0.0/files/p12/p12073186/s52281224.txt'
[Errno 2] No such file or directory: '/datasets/mimic/physionet.org/files/mimic-cxr/2.0.0/files/p12/p12073186/s52281224.txt'
[Errno 2] No such file or directory: '/datasets/mimic/physionet.org/files/mimic-cxr/2.0.0/files/p12/p12073186/s54578036.txt'
[Errno 2] No such file or directory: '/datasets/mimic/physionet.org/files/mimic-cxr/2.0.0/files/p12/p12073186/s54578036.txt'
[Errno 2] No such file or directory: '/datasets/mimic/physionet.org/files/mimic-cxr/2.0.0/files/p12/p12073186/s58887826.txt'
[Errno 2] No such file or directory: '/datasets/mimic/physionet.org/files/mimic-cxr/2.0.0/files/p12/p12073186/s58887826.txt'


100%|██████████| 112413/112413 [06:01<00:00, 310.88it/s]


In [ ]:
create_tsv(val_img_paths, val_train_txts, split="val")

In [ ]:
len(train_img_paths)

In [ ]:
!pip install matplotlib

In [ ]:
from scipy import ndimage
import numpy as np
import matplotlib.pyplot as plt

def show_image(xray_image):
    plt.imshow(xray_image, cmap="gray")

In [ ]:
ds = dicom.dcmread("/datasets/mimic/physionet.org/files/mimic-cxr/2.0.0/files/p10/p10000032/s50414267/02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.dcm")

In [ ]:
show_image(ds.pixel_array)

In [71]:
ds.ViewPosition

'LATERAL'

In [58]:
determine_view_position("/datasets/mimic/physionet.org/files/mimic-cxr/2.0.0/files/p10/p10000032/s50414267/174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962.dcm")

'Lateral'

In [59]:
determine_view_position("/datasets/mimic/physionet.org/files/mimic-cxr/2.0.0/files/p10/p10000032/s50414267/02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.dcm")

'Frontal'